
**AYGAZ GÖRÜNTÜ İŞLEME PROJESİ**

**Hayvan Sınıflandırma İçin CNN Modeli**


**Proje Özeti**
Bu projenin amacı, Konvolüsiyonel Sinir Ağlarının (CNN) temelini anlamak ve hayvan resimlerini sınıflandırmak için bir model oluşturmaktır. Proje, bir CNN modeli eğitme, çeşitli koşullarda test etme ve performansını analiz etmeyi kapsamaktadır. Ayrıca, görüntü manipülasyonu ve renk sabitleme gibi teknikler de uygulanmaktadır.

1.	**Veri Seti**
Bu projede kullanılan veri seti, "Animals with Attributes 2" adlı veri setidir. Veri setine [Kaggle](https://www.kaggle.com/datasets/rrebirrth/animals-with-attributes-2) üzerinden erişebilirsiniz. Kullanılan resimler `JPEGImages` klasöründe yer almaktadır.
Model, aşağıdaki on hayvan sınıfını sınıflandıracak şekilde tasarlanmıştır:


	* Collie
	* Dolphin
    * Elephant
    * Fox
    * Moose
    * Rabbit
    * Sheep
    * Squirrel
    * Giant Panda
    * Polar Bear

**Gerekli Kütüphaneler Hakkında Bilgi**
* **os:** Dosya ve dizin işlemleri yapmanızı sağlar. Örneğin, dosya yolu birleştirme ve dizin listeleme.
* **shutil:** Dosya ve dizinleri kopyalamak, taşımak ve silmek için kullanılır.
* **cv2 (OpenCV):** Görüntü işleme için kullanılır, resimleri okuma, boyutlandırma, kenar tespiti gibi işlemler yapar.
* **numpy (np):** Matematiksel işlemler ve büyük veri setleriyle çalışmak için kullanılan bir kütüphanedir.
* **sklearn.model_selection.train_test_split:** Veriyi eğitim ve test setlerine ayırmak için kullanılır.
* **tensorflow.keras.models.Sequential:** Katmanları sırasıyla ekleyerek derin öğrenme modeli oluşturmanıza olanak tanır.
* **tensorflow.keras.layers.Conv2D:** Görüntülerdeki özellikleri öğrenmek için konvolüsyonel katman ekler.
* **tensorflow.keras.layers.MaxPooling2D:** Görüntülerin boyutlarını küçültür ve önemli özelliklerin korunmasını sağlar.
* **tensorflow.keras.layers.Flatten:** Çok boyutlu veriyi tek boyutlu hale getirir.
* **tensorflow.keras.layers.Dense:** Tam bağlantılı katman ekler ve modelin karar verme kısmını oluşturur.
* **tensorflow.keras.layers.Dropout:** Aşırı uyumu (overfitting) engellemek için bazı nöronları rastgele sıfırlar.
* **tensorflow.keras.layers.Input:** Modelin giriş boyutunu tanımlar.
* **tensorflow.keras.preprocessing.image.ImageDataGenerator:** Veri artırma (augmentation) işlemleri yaparak eğitim verisini çeşitlendirir.
* **tensorflow.keras.optimizers.Adam:** Eğitim sırasında modelin parametrelerini optimize etmek için kullanılan bir algoritmadır.
* **sklearn.metrics.classification_report:** Modelin performansını doğruluk, precision, recall gibi metriklerle özetler.
* **sklearn.metrics.confusion_matrix:** Modelin doğru ve yanlış sınıflandırmalarını görselleştiren bir matris oluşturur.
* **seaborn (sns):** İstatistiksel veriyi görselleştirmek için kullanılan bir grafik kütüphanesidir.
* **matplotlib.pyplot (plt):** Grafikler ve çizimler oluşturmak için kullanılan bir kütüphanedir.
* **sklearn.preprocessing.OneHotEncoder:** Kategorik verileri sayısal verilere dönüştürür, her sınıf için ikili vektörler oluşturur.



In [1]:
#Gerekli kütüphaneler
import os
import shutil
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder


**Verinin Yüklenmesi ve Filtrelenmesi**
10 sınıfa ait görsellerin bulunduğu klasörler belirlenir ve yalnızca seçilen sınıflara ait görüntülerin bulunduğu klasörler başka klasöre kopyalanır. Ayrıca her sınıf için yalnızca ilk 650 görüntü saklanır, bu sayede veri setinin boyutu dengelenmiş olur.

In [2]:
# JPEGImages klasörü
data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"

# Sınıfları listeleme
classes = os.listdir(data_dir)
print(f"Toplam sınıf sayısı: {len(classes)}")
print("Sınıflar:")
print(classes)

# Filtrelenen veri setinin saklanacağı klasör
filtered_dir = "/kaggle/working/filtered_animals"
os.makedirs(filtered_dir, exist_ok=True)

# Kullanılacak sınıflar
selected_classes = ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant+panda', 'polar+bear']


Toplam sınıf sayısı: 50
Sınıflar:
['fox', 'wolf', 'horse', 'antelope', 'hamster', 'skunk', 'chimpanzee', 'lion', 'otter', 'giant+panda', 'raccoon', 'hippopotamus', 'bobcat', 'pig', 'rat', 'spider+monkey', 'buffalo', 'mouse', 'tiger', 'bat', 'grizzly+bear', 'gorilla', 'dalmatian', 'killer+whale', 'siamese+cat', 'humpback+whale', 'chihuahua', 'beaver', 'polar+bear', 'german+shepherd', 'elephant', 'sheep', 'collie', 'moose', 'zebra', 'seal', 'cow', 'ox', 'mole', 'rabbit', 'giraffe', 'persian+cat', 'rhinoceros', 'dolphin', 'blue+whale', 'squirrel', 'leopard', 'deer', 'weasel', 'walrus']


In [3]:
#Sadece gerekli olan sınıfları kopyalayarak yeni bir klasör oluşturuyoruz.

for class_name in selected_classes:
    source_path = os.path.join(data_dir, class_name)
    target_path = os.path.join(filtered_dir, class_name)
    if os.path.exists(source_path):
        shutil.copytree(source_path, target_path)
        print(f"{class_name} sınıfı kopyalandı.")

collie sınıfı kopyalandı.
dolphin sınıfı kopyalandı.
elephant sınıfı kopyalandı.
fox sınıfı kopyalandı.
moose sınıfı kopyalandı.
rabbit sınıfı kopyalandı.
sheep sınıfı kopyalandı.
squirrel sınıfı kopyalandı.
giant+panda sınıfı kopyalandı.
polar+bear sınıfı kopyalandı.


In [4]:
#Resim sayısını dengelemek için her sınıftan yalnızca ilk 650 resmi saklıyoruz.
for class_name in selected_classes:
    class_path = os.path.join(filtered_dir, class_name)
    images = sorted(os.listdir(class_path))  # Alfabetik sıraya göre sıralama
    for image in images[650:]:  # İlk 650 resmi tut
        os.remove(os.path.join(class_path, image))
    print(f"{class_name} sınıfından yalnızca ilk 650 resim tutuldu.")

collie sınıfından yalnızca ilk 650 resim tutuldu.
dolphin sınıfından yalnızca ilk 650 resim tutuldu.
elephant sınıfından yalnızca ilk 650 resim tutuldu.
fox sınıfından yalnızca ilk 650 resim tutuldu.
moose sınıfından yalnızca ilk 650 resim tutuldu.
rabbit sınıfından yalnızca ilk 650 resim tutuldu.
sheep sınıfından yalnızca ilk 650 resim tutuldu.
squirrel sınıfından yalnızca ilk 650 resim tutuldu.
giant+panda sınıfından yalnızca ilk 650 resim tutuldu.
polar+bear sınıfından yalnızca ilk 650 resim tutuldu.


### Etiketler ve Resimler İçin Listeler Oluşturulması
Bu bölümde, görsellerin ve etiketlerin listelendiği ve işlenmeye hazır hale getirildiği adımlar yer almaktadır.

1. **Boş listeler oluşturuluyor**:
   - `X` listesi, görselleri depolamak için kullanılır.
   - `y` listesi, her bir görselin etiketini (sınıf ismi) saklamak için kullanılır.

2. **Her sınıf için resimler okunuyor**:
   - `os.listdir(filtered_dir)` komutu ile `filtered_dir` dizinindeki tüm alt dizinler (sınıflar) liste haline getirilir.
   - Her bir sınıf için, o sınıfa ait resimler `cv2.imread` ile okunur.
   - Görseller, `cv2.resize(img, (128, 128))` fonksiyonu ile 128x128 piksel boyutuna yeniden boyutlandırılır.
   - Görsellerin değerleri `[0, 1]` aralığına normalize edilir (`img_normalized = img_resized / 255.0`).
   - Görsel ve etiketler sırasıyla `X` ve `y` listelerine eklenir.

3. **X ve y NumPy Dizilerine Dönüştürülür**:
   - `X` ve `y` listeleri NumPy dizilerine dönüştürülür (`np.array(X)` ve `np.array(y)`).

4. **Eğitim ve Test Verilerine Ayırma**:
   - `train_test_split` fonksiyonu ile veriler eğitim ve test setlerine ayrılır. Bu işlemde:
     - `test_size=0.2` ile verilerin %20'si test verisi olarak ayrılır.
     - `random_state=42` parametresi, her çalıştırmada aynı sonuçların elde edilmesini sağlar.
   - Eğitim seti ve test setinin boyutları yazdırılır.

Sonuç olarak, bu kod, veri setini eğitim ve test setlerine ayırarak derin öğrenme modelini eğitmek için kullanılabilir hale getirir.

In [5]:
# Etiketler ve resimler için listeler
X = []  # Görüntü verisi
y = []  # Etiketler (sınıflar)

# Her sınıf için resimleri oku
for class_name in os.listdir(filtered_dir):
    class_path = os.path.join(filtered_dir, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img = cv2.imread(image_path)
            if img is not None:
                img_resized = cv2.resize(img, (128, 128))  # Görüntüyü yeniden boyutlandır
                img_normalized = img_resized / 255.0  # Normalize et
                X.append(img_normalized)
                y.append(class_name)  # Etiket olarak sınıf ismini kullan

# X ve y numpy dizilerine dönüştürülüyor
X = np.array(X)
y = np.array(y)

# Eğitim ve test verilerini ayırma (80% eğitim, 20% test)
# X: images, y: labels anlamına gelmektedir
# random_state=42 her çalıştırmada aynı sonuçları almamızı sağlar.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim verisi boyutu: {X_train.shape}")
print(f"Test verisi boyutu: {X_test.shape}")

Eğitim verisi boyutu: (5200, 128, 128, 3)
Test verisi boyutu: (1300, 128, 128, 3)


Aşağıdaki kod, görüntü işleme ve etiket dönüştürme işlemleri gerçekleştirir. İlk olarak, ImageDataGenerator sınıfı ile veri artırma işlemleri yapılır; bu işlemler arasında döndürme, kaydırma, yakınlaştırma ve yatay çevirme gibi teknikler bulunur. Bu sayede modelin genelleme yeteneği artırılır. Ayrıca, özel bir custom_augmentation fonksiyonu ile görüntülere bulanıklaştırma ve kenar bulma gibi manipülasyonlar uygulanabilir. Son olarak, etiketler, OneHotEncoder kullanılarak sayısal vektörlere dönüştürülür, bu da modelin sınıflandırma işlemi için gereklidir. Bu adımlar, modelin eğitim verisi üzerinde daha iyi performans göstermesine yardımcı olur.

In [9]:
# Veri artırma işlemleri için ImageDataGenerator oluşturma
datagen = ImageDataGenerator(
    rotation_range=20,         # Görüntüleri rastgele döndürme
    width_shift_range=0.1,     # Yatayda kaydırma
    height_shift_range=0.1,    # Dikeyde kaydırma
    shear_range=0.2,           # Kesme dönüşümü
    zoom_range=0.2,            # Yakınlaştırma
    horizontal_flip=True,      # Yatayda çevirme
    fill_mode='nearest',       # Boş alanları doldurmak için 'nearest' (yakın olan değeri kullanma)
    rescale=1./255             # Görüntüleri normalize etme (0-255 arası değerleri 0-1 arası normalize eder)
)

# Özelleştirilmiş veri artırma fonksiyonu
def custom_augmentation(img):
    # Bulanıklaştırma
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    
    # Kenar bulma (Canny Edge Detection)
    edge_img = cv2.Canny(blurred_img, 100, 200)

# Etiketleri sayısal değerlere dönüştürme
# OneHotEncoder oluştur
one_hot_encoder = OneHotEncoder(sparse_output=False)  # sparse_output=False ile dönen vektör numpy array olur

# Eğitim ve test etiketlerini dönüştür
y_train_encoded = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1))
y_test_encoded = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1))


Bu kod, bir konvolüsyonel sinir ağı (CNN) modeli oluşturur ve eğitir. Model, 128x128 boyutlarında 3 kanallı görüntüleri alacak şekilde yapılandırılmıştır. İlk iki katmanda, her biri Conv2D ile filtre uygulayan ve ardından MaxPooling2D ile boyutlarını küçülten katmanlar bulunur. Ardından, Flatten katmanı ile veriler tek boyutlu hale getirilir ve Dense katmanında 128 nöronlu bir tam bağlantılı katman kullanılır. Son katmanda, sınıf sayısına göre softmax aktivasyon fonksiyonu ile çok sınıflı sınıflandırma yapılır. Model, Adam optimizasyon algoritması ve categorical_crossentropy kayıp fonksiyonu ile derlenir. Eğitim sırasında, veri artırma teknikleri kullanılarak modelin genelleme yeteneği artırılır ve model 10 epoch boyunca eğitilir, ardından doğrulama verisi ile test edilir.

In [11]:
# CNN Modeli tanımlama
model = Sequential([
    Input(shape=(128, 128, 3)),  # Input katmanını ekledik
    Conv2D(32, (3, 3), activation='relu'),  
    MaxPooling2D(pool_size=(2, 2)),
      
    Conv2D(64, (3, 3), activation='relu'), 
    MaxPooling2D(pool_size=(2, 2)),
      
    Flatten(),
    Dense(128, activation='relu'),  
    Dropout(0.3),  
    Dense(len(np.unique(y)), activation='softmax')  # Çıkış katmanı, sınıf sayısına göre softmax
])

# Belirli bir öğrenme oranı ile Adam optimizer kullanmak
learning_rate = 0.001  # İstediğimiz öğrenme oranını buraya yazabiliriz
optimizer = Adam(learning_rate=learning_rate)

# Modeli derleme

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim verisini kullanarak veri artırma uygulama
datagen.fit(X_train)

# Veri artırma işlemi sırasında, görüntüleri sadece eğitim verisine uygulayacağız.

# Modeli eğitme
history = model.fit(
    datagen.flow(X_train, y_train_encoded, batch_size=32),  # y_train_encoded kullanıyoruz
    epochs=10,
    validation_data=(X_test, y_test_encoded)  # y_test_encoded kullanıyoruz 
)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 74s 435ms/step - accuracy: 0.1035 - loss: 2.3031 - val_accuracy: 0.0908 - val_loss: 19.2113
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 78s 473ms/step - accuracy: 0.0976 - loss: 2.3026 - val_accuracy: 0.0908 - val_loss: 19.3779
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 423ms/step - accuracy: 0.1065 - loss: 2.3025 - val_accuracy: 0.0908 - val_loss: 19.5247
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 69s 416ms/step - accuracy: 0.0898 - loss: 2.3027 - val_accuracy: 0.0908 - val_loss: 19.5348
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 422ms/step - accuracy: 0.0989 - loss: 2.3024 - val_accuracy: 0.0908 - val_loss: 19.4484
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 421ms/step - accuracy: 0.0978 - loss: 2.3029 - val_accuracy: 0.0908 - val_loss: 19.6306
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 68s 414ms/step - accuracy: 0.1033 - loss: 2.3024 - val_accuracy: 0.0908 - val_loss: 19.8128
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 69s 420ms/step - accuracy: 0.1000 - 

Yukarıdaki çıktılar, modelin öğreniminde ciddi sorunlar olduğunu ve modelin doğru şekilde öğrenemediğini gösteriyor.
-Eğitim doğruluğu (accuracy) her epoch boyunca %10 civarında ve artmıyor.
-Eğitim kaybı (loss) ise yaklaşık 2.302'de sabit kalmış durumda.
-Doğrulama doğruluğu (val_accuracy) da her epoch boyunca %9 civarında ve sabit.
-Doğrulama kaybı (val_loss) ise aşırı yüksek (örneğin, 19.2113'ten başlayarak artmaya devam ediyor).
Bu durum, modelin sınıflandırma yapmayı öğrenemediğini ve rastgele tahminler yaptığını gösterir. y_train_encoded veya y_test_encoded doğru bir şekilde kategorik hale getirilmemiş olabilir. OneHotEncoder doğru uygulanmamış olabilir. X_train ve X_test verileri uygun şekilde normalize edilmemiş olabilir (örneğin, tüm piksel değerlerinin [0, 1] aralığına getirilmesi gerekiyor).

In [12]:
# Modelin test edilmesi
results = model.evaluate(X_test, y_test_encoded, verbose=1)
print(f"Test kaybı: {results[0]}")
print(f"Test doğruluğu: {results[1]}")

41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.0962 - loss: 19.4738
Test kaybı: 19.703445434570312
Test doğruluğu: 0.09076923131942749


Yukarıdaki çıktı, modelin test veri seti üzerinde de öğrenemediğini ve sınıflandırma performansının rastgele tahminden farksız olduğunu gösteriyor.

In [13]:
# Manipüle edilmiş görüntüler oluşturma
def get_manipulated_images(images):
    manipulated_images = []
    for img in images:
        brightness_factor = np.random.uniform(0.5, 1.5)
        manipulated_img = np.clip(img * brightness_factor, 0, 1)  # Parlaklık değişimi
        manipulated_images.append(manipulated_img)
    return np.array(manipulated_images)

# Manipüle edilmiş test seti
X_test_manipulated = get_manipulated_images(X_test)

# Manipüle edilmiş set ile model testi
manipulated_results = model.evaluate(X_test_manipulated, y_test_encoded, verbose=1)
print(f"Manipüle edilmiş test kaybı: {manipulated_results[0]}")
print(f"Manipüle edilmiş test doğruluğu: {manipulated_results[1]}")


41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step - accuracy: 0.0962 - loss: 19.0088
Manipüle edilmiş test kaybı: 19.265670776367188
Manipüle edilmiş test doğruluğu: 0.09076923131942749


Bu sonuç, modelin manipüle edilmiş (örneğin, parlaklık değişimi uygulanmış) görüntüler üzerinde de kötü performans gösterdiğini, öğrenme başarısızlığının manipülasyonlarla iyileşmediğini gösteriyor. Normal test doğruluğu ile aynı (%9 civarında). Bu, modelin manipüle edilmiş görüntülerde de rastgele tahmin düzeyinde olduğunu ve manipülasyonların modeli geliştirmediğini doğruluyor.

In [14]:
# Renk sabitliği (Gray World algoritması)
def get_wb_images(images):
    wb_images = []
    for img in images:
        avg_color = np.mean(img, axis=(0, 1))
        wb_img = np.clip(img * (np.mean(avg_color) / avg_color), 0, 1)  # Normalize et
        wb_images.append(wb_img)
    return np.array(wb_images)

# Renk sabitliği uygulanmış test seti
X_test_wb = get_wb_images(X_test_manipulated)

# Renk sabitliği uygulanmış set ile model testi
wb_results = model.evaluate(X_test_wb, y_test_encoded, verbose=1)
print(f"Renk sabitliği uygulanmış test kaybı: {wb_results[0]}")
print(f"Renk sabitliği uygulanmış test doğruluğu: {wb_results[1]}")



41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.0962 - loss: 24.4342
Renk sabitliği uygulanmış test kaybı: 24.637224197387695
Renk sabitliği uygulanmış test doğruluğu: 0.09076923131942749


Bu sonuç, renk sabitliği (Gray World algoritması) uygulanmış veri üzerinde modelin performansının yine oldukça kötü olduğunu, test kaybının arttığını ve doğruluğun rastgele tahmin seviyesinde (%9) kaldığını gösteriyor. 

24.6372 -> Bu değer, hem orijinal test kaybı (19.7034) hem de manipüle edilmiş test kaybından (19.2656) daha yüksek. Bu, renk sabitliğinin modele zarar verdiğini ve öğrenmeyi daha da kötüleştirdiğini gösteriyor.


**Sonuçların Karşılaştırması**

                             
**Orijinal Test Seti ->**	           Test Kaybı (Loss): 19.7034	     Test Doğruluğu (Accuracy): 0.0907 (%9)
**Manipüle Edilmiş Test Seti ->**	   Test Kaybı (Loss): 19.2656	     Test Doğruluğu (Accuracy): 0.0907 (%9)
**Renk Sabitliği Test Seti  ->**	   Test Kaybı (Loss): 24.6372	     Test Doğruluğu (Accuracy): 0.0907 (%9)

**Karşılaştırma ve Yorum**

*Doğruluk:*

Üç test durumunda da doğruluk %9 seviyesinde ve bu, modelin sınıfları rastgele tahmin ettiğini gösteriyor. Yani model sınıflar arasındaki özellikleri öğrenememiş.

*Kaybın (Loss) Değişimi:*

Manipüle edilmiş görüntülerde kayıp bir miktar düşmüş olsa da bu, doğruluğa yansımamış. Renk sabitliği uygulandığında ise kayıp önemli ölçüde artmış.
Bu durum, manipülasyon ve renk sabitliği uygulamalarının modelin öğrenme yeteneğini daha da zayıflattığını gösteriyor.

*Genel Performans:*

Model, herhangi bir veri setinde işe yarar bir şekilde öğrenememiş. Hem eğitim sırasında hem de testlerde başarısız sonuçlar alınmış.

**Çözüm önerileri:**

- Hiperparametre ayarı yapabiliriz.
- Transfer Learning Kullanabiliriz.
- Model mimarisini geliştirebiliriz.